# 14-1 sub-chain

目标例子：

- 如果输入是中文，使用ChatZhipuAI回答
- 其他，使用ChatOllama回答

## (1) 将一个函数转换成 runnable的函数
1. @chain装饰器

```python
from langchain_core.runnables import chain
```
  
2. RunnableLambda()

```python
from langchain_core.runnables import RunnableLambda
```
runnable_func为普通函数，但是经过@chain/RunnableLambda包装
```python
chain = prompt | chat_model | runnable_func
```

## (2) 中文字符判断

```python
import re
text = "这里有中文～～～"
pattern = re.compile(r'[\u4e00-\u9fff]')
pattern.search(text)
```

In [1]:
import re

# {"question":"xxxxx"}
def contains_chinese(info):
    text = info.get("question", "")
    pattern = re.compile(r'[\u4e00-\u9fff]')
    # re.Match
    is_chinese = bool(pattern.search(text))
    if is_chinese:
        return "chinese"
    else:
        return "other"

In [3]:
contains_chinese({"question":"what is your name?"})

'other'

In [4]:
import os
import getpass

os.environ["ZHIPUAI_API_KEY"] =  getpass.getpass("请输入ZHIPUAI_API_KEY...")

请输入ZHIPUAI_API_KEY... ········


In [6]:
# ChatPromptTempalte
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama, ChatZhipuAI
from langchain_core.output_parsers import StrOutputParser

zhipuai_chain = ChatPromptTemplate.from_template(
    """你是一个中文专家。回答问题时，你总是以`大小寒告诉我的:`开头。
请回答下面的问题：
问题：{question}
回答：""") | ChatZhipuAI(model="glm-4")

ollama_chain = ChatPromptTemplate.from_template(
    """你是一个非中文专家。回答问题时，你总是以`苏格拉底告诉我的:`开头。
请回答下面的问题：
问题：{question}
回答：""") | ChatOllama(model="llama3")


In [14]:
from langchain_core.runnables import chain

@chain
def route(info):
    # print(f"info:{info}")
    # return "随便写一点返回"
    if info["topic"] == "chinese":
        return zhipuai_chain
    else:
        return ollama_chain

In [15]:
from langchain_core.runnables import RunnableLambda

# chain = {"topic": contains_chinese, "question": lambda x: x["question"]} | RunnableLambda(route)
chain = {"topic": contains_chinese, "question": lambda x: x["question"]} | route
result = chain.invoke({"question": "什么是langchain?"})
result

AIMessage(content='大小寒告诉我的：LangChain 是一种基于语言模型的连锁推理框架，它能够处理较长的文本序列，通过将文本分解成较短的片段，再利用语言模型对这些片段进行推理和链接，以此来完成复杂的文本生成或理解任务。这种方法可以有效地避免因为文本长度超出模型处理能力而导致的性能下降问题，提高模型在处理长文本时的效果。LangChain 框架适用于多种语言模型，并能应用于多种场景，如问答系统、文章生成、文本摘要等。', response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 37, 'total_tokens': 145}, 'model_name': 'glm-4', 'finish_reason': 'stop'}, id='run-b59f1667-e381-4ffa-ae71-61302da3756e-0')